In [1]:
import numpy as np
import tonic
import tonic.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
#from tonic.dataset import Dataset
from typing import Callable, Optional
import torch
import matplotlib.pyplot as plt
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
import matplotlib.patches as patches
from tqdm.notebook import tqdm

In [2]:
import sys
sys.path.append("./mnist_sg_cnn")

In [3]:
import utils
import snn_utils
import base_model
import lenet_decolle_model

In [4]:
[torch.cuda.device(i) for i in range(torch.cuda.device_count())]

In [5]:
dir = "/media/user/EVIMO/raw/imo/eval/scene15_dyn_test_01/left_camera/ground_truth_000000"
sensor_size = [640, 480, 2]
batch_size = 4
num_bins_per_frame = 10
framerate = 200
device = 'cuda'
epochs=40

In [6]:
class EVIMO(Dataset):
    def __init__(self,
                 dir: str,
                 item_to_find: int,
                 num_bins_per_frame: int,
                 #start_idx: int,
                ):
        self.dir = dir
        self.item_to_find = item_to_find
        self.num_bins_per_frame = num_bins_per_frame
        self.length = np.load(self.dir + "/length.npy")
        print(self.length)
        #self.start_idx = start_idx


    def __getitem__(self, index):
        item = np.load(self.dir + "/" + str(index) + ".npy", allow_pickle=True).tolist()
        if self.item_to_find in item["objs_in_mask"]:
            target = np.asarray([0, 1])
        else:
            target = np.asarray([1, 0])

        target = np.tile(target, (self.num_bins_per_frame, 1))

        return {"data": item["binned_events"], "target": target}

    def __len__(self) -> int:
        return self.length # - self.start_idx
    
    

In [7]:
tonic_dataset = EVIMO(dir="./data/EVIMO/left_cam/scene13_test5", item_to_find=23, num_bins_per_frame=num_bins_per_frame)

477


In [8]:
1-0.2972972972972973

0.7027027027027026

In [9]:
#tonic_dataset[0]

In [10]:
batched_trainloader = DataLoader(tonic_dataset, batch_size=batch_size, shuffle=False) # collate_fn=tonic.collation.PadTensors(),
#single_trainloader = DataLoader(tonic_dataset, batch_size=1, shuffle=False)

In [11]:
#for i_batch, sample_batched in enumerate(trainloader):
#    print(i_batch, sample_batched['data'].size(), sample_batched['target'].size())

In [12]:
datait = iter(batched_trainloader)
batched_data = next(datait)

In [13]:
batched_data['data'].shape

torch.Size([4, 10, 2, 480, 640])

In [14]:
batched_data['target'].shape

torch.Size([4, 10, 2])

In [15]:
item = next(iter(batched_trainloader))
data = item['data']
#target = item['target']

def decolle_loss(r, s, tgt):
    loss_tv = 0
    for i in range(len(r)):
        #print(f"Loss Readout shape : {r[i].shape}")
        #print(f"Loss Target shape : {tgt.shape}")
        loss_tv += loss(r[i],tgt) 
    return loss_tv

loss = torch.nn.SmoothL1Loss()

convnet_sg = lenet_decolle_model.LenetDECOLLE(out_channels=2,
                    Nhid=[32, 64, 128, 256], #Number of convolution channels
                    Mhid=[32, 64, 128],
                    kernel_size=[4, 8, 16, 32],
                    pool_size=[2, 4, 4, 8],
                    input_shape=[2, 480, 640],  # data.shape[1:],
                    alpha=[.95],
                    alpharp=[.65],
                    beta=[.92],
                    num_conv_layers=4,
                    num_mlp_layers=3,
                    lc_ampl=0.5).to(device)

#net = lenet_decolle_model.LenetDECOLLE(Nhid=[1,8],Mhid=[32,64],out_channels=2, input_shape=[2, 480, 640]).to(device)

#convnet_sg

data_d = data.to(device)
#target_d = target.to(device)
convnet_sg.init_parameters(data_d) # Modifies readout dim

LAYER SIZE: 2439712
STDV: 0.00032011109219189804
LAYER SIZE: 298304
STDV: 0.0009154623036203756
LAYER SIZE: 34048
STDV: 0.002709718654762875
LAYER SIZE: 512
STDV: 0.022097086912079608
LAYER SIZE: 32
STDV: 0.08838834764831843
LAYER SIZE: 64
STDV: 0.0625
LAYER SIZE: 128
STDV: 0.044194173824159216


In [16]:
data_d.shape

torch.Size([4, 10, 2, 480, 640])

In [17]:
#data_d.transpose(0, 1).shape

In [18]:
#data.shape[1:]

In [19]:
#target_d

In [20]:
#target_d.shape

In [22]:
opt_conv = torch.optim.Adamax(convnet_sg.get_trainable_parameters(), lr=1e-4, betas=[0., .95])

for e in range(epochs):        
    error = []
    accuracy=[]
    for item in tqdm(iter(batched_trainloader), desc=f"Epoch {e}"):
        data = item['data']
        label = item['target']

        if label.shape[0] != 4:
            #print(f"last entry")
            break
        
        convnet_sg.train()
        loss_hist = 0
        data_d = data.to(device)
        label_d = label.to(device)
        convnet_sg.init(data_d, burnin=2)
        readout = 0

        data_d = data_d.transpose(0, 1)
        label_d = label_d.transpose(0, 1)
        
        for n in range(num_bins_per_frame):
            st, rt, ut = convnet_sg.forward(data_d[n])   
            #print(rt[0].shape)
            #print(label_d.shape)
            loss_tv = decolle_loss(rt, st, label_d[n])
            
            loss_tv.backward()
            opt_conv.step()
            opt_conv.zero_grad()
            loss_hist += loss_tv
            readout += rt[-1]

        #print(f"Readout: {readout}")
        #error += (readout.argmax(axis=1)!=label_d.argmax(axis=1)).float()
        #accuracy+=(readout.argmax(axis=1)==label_d.argmax(axis=1)).float()

        error += (readout.argmax(axis=1)!=label_d[-1].argmax(axis=1)).float()
        accuracy+=(readout.argmax(axis=1)==label_d[-1].argmax(axis=1)).float()
        
    print('Training Error', torch.mean(torch.Tensor(error)).data)
    print('Training accuracy', torch.mean(torch.Tensor(accuracy)).data)     
    print('Epoch', e, 'Loss', loss_hist.data)
    PATH = './EVIMO_class_cnn.pth'  
    torch.save(convnet_sg.state_dict(), PATH)

  0%|          | 0/120 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
type(loss_tv)

In [ ]:
error = []
accuracy=[]
y_pred = []
y_true = []
#convnet_sg.requires_init = True
for item in iter(batched_trainloader):
    #print("Item")
    #print(item)
    data = item['data']
    label = item['target']
    if label.shape[0] != 4:
        #print(f"last entry: {label}")
        break

    #print("earliest label")
    #print(label)
    #print(label.shape)

    
    loss_hist = 0
    data_d = data.to(device)
    label_d = label.to(device)
    print(f"data_d shape: {data_d.shape}")
    print(f"data_d_tranpose shape: {data_d.transpose(0,1).shape}")
    print(f"Len convnet: {len(convnet_sg)}")
    print(f"data init shape: {data_d.transpose(0,1)[:, 0, :, :].shape}")
    print(f"Test data shape: {data_d[0].shape}")
    
    convnet_sg.init_evimo(data_d)
    readout = 0
    with torch.no_grad():
        #for i in range(0, len(data_d)):
        #d = data_d[i:i+1]
        
        #print(d.shape)
        
        st, rt, ut = convnet_sg.forward(data_d)

        
        
        print(f"Label shape: {label_d.shape}")
        print("Readout")
        print(rt)
        print(rt[0].shape)

        #print("labels")
        #print(label_d)
        #print(label_d.shape)
        
        
        loss_tv = decolle_loss(rt, st, label_d)
        
        
        loss_hist += loss_tv
        readout += rt[-1]
        print(f"Readout: {readout}")
        output = (readout.argmax(axis=1)).data.cpu().numpy()
        print(f"Output: {output}")
        y_pred.extend(output)
        #labels = label_d.argmax(axis=1)
        print(f"Labels: {label_d}")
        labels = (label_d.argmax(axis=1)).data.cpu().numpy()
        print(f"Labels: {labels}")
        y_true.extend(labels)
        accuracy+=(readout.argmax(axis=1)==label_d.argmax(axis=1)).float()
        error += (readout.argmax(axis=1)!=label_d.argmax(axis=1)).float()
        break
        
print('Testing Error', torch.mean(torch.Tensor(error)).data)
print('Testing accuracy', torch.mean(torch.Tensor(accuracy)).data)

In [ ]:
print(y_true)
print(y_pred)

In [ ]:
"""
convnet_sg = lenet_decolle_model.LenetDECOLLE( out_channels=2,
                    Nhid=[16,32], #Number of convolution channels
                    Mhid=[64],
                    kernel_size=[7],
                    pool_size=[2,2],
                    input_shape=[2, 480, 640],  # data.shape[1:],
                    alpha=[.95],
                    alpharp=[.65],
                    beta=[.92],
                    num_conv_layers=2,
                    num_mlp_layers=1,
                    lc_ampl=.5).to(device)

"""

In [ ]:
net = lenet_decolle_model.LenetDECOLLE(Nhid=[16,32],Mhid=[64],out_channels=2, pool_size=[2,2], num_conv_layers=2, num_mlp_layers=1, lc_ampl=.5, input_shape=[2, 480, 640])
d = torch.zeros([1, 2, 480, 640])
print(d.shape)
st, rt, ut = net.forward(d)
print(rt[0].shape)

In [ ]:
print(rt)

In [ ]:
target = torch.zeros([1, 2])
target

In [ ]:
loss_tv = decolle_loss(rt, st, target)